## Create same train/test sets as YoonKim


In [ ]:
import pickle
import numpy as np


def get_idx_from_sent(sent, word_idx_map, max_l=56, k=300, filter_h=5):
    """
    Transforms sentence into a list of indices. Pad with zeroes.
    """
    x = []
    pad = filter_h - 1
    for i in range(pad):
        x.append(0)
    words = sent.split()
    for word in words:
        if word in word_idx_map:
            x.append(word_idx_map[word])
    while len(x) < max_l+2*pad:
        x.append(0)
    return x

def make_idx_data(data, word_idx_map, max_l=56, k=300, filter_h=5):
    """
    Transforms sentences into a 2-d matrix.
    """
    X, y = [], []
    for row in data:
        sent = get_idx_from_sent( row['text'], word_idx_map, max_l, k, filter_h)
        X.append( sent )
        y.append( row['y'] )
    X = np.array( X, dtype="int" )
    y = np.array( y, dtype="int" )
    return X, y

# train = [ { y: 'class', text: 'sentence'} ]
# dev = [ { y: 'class', text: 'sentence'} ]
# test = [ { y: 'class', text: 'sentence'} ]

# W = google pretrained word2vec matrix, each column represent a word
# W2 = random generated word2vec matrix, each column represent a word

# word_idx_map: word => column number in word2vec matrix
# vocab = set of all words in dataset


# alterative datasources: data/MR.p, data/SST1.p, data/SST2.p
data = pickle.load(open("data/MR.p","rb"))
train, dev, test, W, W2, word_idx_map, vocab = data[0], data[1], data[2], data[3], data[4], data[5], data[6]

print( len(train), len(dev), len(test), W.shape, W2.shape, len(word_idx_map), len(vocab) )
print( "data loaded!" )

In [ ]:
from sklearn import preprocessing
W3 = preprocessing.normalize(W, norm='l2')

In [ ]:
import keras
import os

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]=""

In [ ]:
from keras.layers import Input, Dense, Flatten, Embedding
from keras.models import Model
from keras.layers import Conv1D, GlobalMaxPooling1D, Input, Add, Dropout
from keras.layers.merge import Concatenate

np.random.seed(2017)

assert W.shape == W2.shape == W3.shape
vocab_size, vec_size = W.shape
sent_length = 64 # train_X.shape[1]

num_filters = 100
kernel_sizes = [3,4,5]
batch_size = 50
dropout_rate = 0.5
l2_constraint = 3

# TODO: check all layer initializers

def conv(x):
    cs = []
    for kernel_size in kernel_sizes:
        c = Conv1D( filters=num_filters, 
          kernel_size=kernel_size, 
          padding="valid", # i.e. no additional padding
          activation="relu",
          strides=1)(x)
        c = GlobalMaxPooling1D()(c)
        cs.append( c )
    return Concatenate()(cs)

def get_model( static=True, initial_embedding=W ):
    inputs = Input(shape=(64,))
    x = Embedding( input_dim=vocab_size, 
              output_dim=vec_size, 
              input_length=sent_length, 
              weights=[ initial_embedding ], 
              trainable=(not static) )(inputs)
    x = conv( x )
    predictions = Dense(1, kernel_initializer='normal', 
            #kernel_regularizer=regularizers.l2(3.), 
            activation='sigmoid')(x)
    model = Model( inputs=inputs, outputs=predictions )
    model.compile( optimizer='Adadelta',
              loss='binary_crossentropy',
              metrics=['binary_accuracy'] )
    return model

def multi_chnnel_conv( x1, x2 ):
    cs = []
    for kernel_size in kernel_sizes:
        shared_conv = Conv1D( filters=num_filters, 
          kernel_size=kernel_size, 
          padding="valid",
          activation="relu",
          strides=1)
        
        c1 = shared_conv( x1 )
        c2 = shared_conv( x2 )
        
        c1 = GlobalMaxPooling1D()(c1)
        c2 = GlobalMaxPooling1D()(c2)
        
        c = Add()( [c1, c2] )
        cs.append( c )
    return Concatenate()(cs)

def get_multi_channel_model( initial_embedding=W ):
    inputs = Input(shape=(64,))
    x_static = Embedding( input_dim=vocab_size, 
              output_dim=vec_size, 
              input_length=sent_length, 
              weights=[ inital_embedding ], 
              trainable=False )(inputs)
    
    x_non_static = Embedding( input_dim=vocab_size, 
              output_dim=vec_size, 
              input_length=sent_length, 
              weights=[ initial_embedding ], 
              trainable=True )(inputs)
    
    x = multi_chnnel_conv( x_static, x_non_static )
    predictions = Dense(1, kernel_initializer='normal', 
            #kernel_regularizer=regularizers.l2(3.), 
            activation='sigmoid')(x)
    model = Model( inputs=inputs, outputs=predictions )
    model.compile( optimizer='Adadelta',
              loss='binary_crossentropy',
              metrics=['binary_accuracy'] )
    return model

In [ ]:
train_X, train_y = make_idx_data( train, word_idx_map )
if dev: dev_X, dev_y = make_idx_data( dev, word_idx_map )
if test: test_X, test_y = make_idx_data( test, word_idx_map )

In [ ]:
from sklearn.model_selection import StratifiedKFold

kf = StratifiedKFold(n_splits=10)
splits = kf.split( train_X, train_y )
    
scores = []
for train_indices, val_indices in splits:
    X_cv_train, X_cv_val = train_X[train_indices], train_X[val_indices]
    y_cv_train, y_cv_val = train_y[train_indices], train_y[val_indices]
    
    model = get_model( initial_embedding=W3 )
    model.fit( X_cv_train, y_cv_train, batch_size=50, epochs=25)
    
    score = model.evaluate( X_cv_val, y_cv_val )
    scores.append( score[1] )
print('final cross validation scores: {}'.format(sum(scores)/len(scores)))